In [0]:
#Importing all the ncessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import seaborn as sns
import nltk
import string
import sqlite3 as sql
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import random
from string import punctuation

import warnings
warnings.simplefilter('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


**Load User Profile Data**

From train_triplets.csv

In [0]:
database1 = "Datasets/mxm_dataset.db"
connection1 = sql.connect(database1)
query1 = '''SELECT * FROM lyrics'''
df_lyrics = pd.read_sql(query1,connection1)

In [0]:
database2 = "Datasets/track_metadata.db"
connection2=sql.connect(database2)
query2 = '''SELECT * FROM songs'''
df_metadata= pd.read_sql(query2,connection2)

In [0]:
lyrics_metadata=df_metadata[df_metadata['track_id'].isin(df_lyrics['track_id'].unique())]

In [0]:
#Importing all the ncessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import seaborn as sns
import nltk
import string
import sqlite3 as sql
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import random
from string import punctuation

import warnings
warnings.simplefilter('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#Change Variables

In [0]:
Num=400
user_count=10

#lyrics_metadata=df_metadata

In [0]:
initial_hidden=pd.read_csv("Datasets/year1_test_triplets_hidden.txt",sep="\t")
initial_hidden.columns=['user_id','song_id','freq_song']
initial_hidden


,user_id,song_id,freq_song
0,00007a02388c208ea7176479f6ae06f8224355b3,SOIHOIQ12A8C138593,2
1,00007a02388c208ea7176479f6ae06f8224355b3,SOYIZSN12A6701E0BB,3
2,00007a02388c208ea7176479f6ae06f8224355b3,SODYZAD12A58A7A525,1
3,00007a02388c208ea7176479f6ae06f8224355b3,SOXLWPN12A8C143667,1
4,00007a02388c208ea7176479f6ae06f8224355b3,SOOHJTL12AB0185497,1
...,...,...,...
1368424,ffff07d7d9bb187aa58c7b81b3d3f35e7cf7c0ee,SOETHAV12AB0183107,1
1368425,ffff07d7d9bb187aa58c7b81b3d3f35e7cf7c0ee,SONPDVJ12A67ADA231,1
1368426,ffff07d7d9bb187aa58c7b81b3d3f35e7cf7c0ee,SOKAESA12A8C1410A1,1
1368427,ffff07d7d9bb187aa58c7b81b3d3f35e7cf7c0ee,SOPQLBY12A6310E992,1


In [0]:
initial_visible=pd.read_csv("Datasets/year1_test_triplets_visible.txt",sep="\t")
initial_visible.columns=['user_id','song_id','freq_song']
print(initial_visible.shape)



(1319893, 3)


#Filter Songs Available in Lyrical Metadata

In [0]:
initial_visible=initial_visible[initial_visible['song_id'].isin(lyrics_metadata['song_id'])]
initial_hidden=initial_hidden[initial_hidden['song_id'].isin(lyrics_metadata['song_id'])]

print(initial_visible.shape)
print(initial_hidden.shape)

(695151, 3)
(720623, 3)


In [0]:
def create_popularity_recommendation(train_data, user_id, item_id, n=10):
    #Get a count of user_ids for each unique song as recommendation score
    train_data_grouped = train_data.groupby([item_id]).agg({user_id: 'count'}).reset_index()
    train_data_grouped.rename(columns = {user_id: 'score'},inplace=True)
    
    #Sort the songs based upon recommendation score
    train_data_sort = train_data_grouped.sort_values(['score', item_id], ascending = [0,1])
    
    #Generate a recommendation rank based upon score
    train_data_sort['Rank'] = train_data_sort.score.rank(ascending=0, method='first')
    
     #Get the top n recommendations
    popularity_recommendations = train_data_sort.head(n)
    return popularity_recommendations



In [0]:
popular_visible = create_popularity_recommendation(initial_visible,'user_id','song_id', Num).reset_index(drop=True)
popular_hidden= create_popularity_recommendation(initial_hidden,'user_id','song_id', Num).reset_index(drop=True)

Randomly Select 1000 Songs from Popular

In [0]:
visible_sample=popular_visible.sample(int(Num/2),random_state=576) 
visible_sample.shape

(200, 3)

In [0]:
selected_visible_Songs=visible_sample['song_id'].unique()
not_selected_visible_songs=popular_hidden[~popular_hidden['song_id'].isin(selected_visible_Songs)]['song_id']

In [0]:
not_selected_visible_songs.shape

(213,)

In [0]:
df_valid_visible=initial_visible[initial_visible['song_id'].isin(selected_visible_Songs)]
display(df_valid_visible.shape)

(79843, 3)

In [0]:
df_valid_hidden=initial_hidden[initial_hidden['song_id'].isin(not_selected_visible_songs)]
display(df_valid_hidden.shape)

(81519, 3)

In [0]:
unique_users_visible=df_valid_visible['user_id'].unique()
unique_songs_visible=df_valid_visible['song_id'].unique()
unique_users_hidden=df_valid_hidden['user_id'].unique()
unique_songs_hidden=df_valid_hidden['song_id'].unique()

print(len(unique_songs_visible))
print(len(unique_songs_hidden))

200
213


In [0]:
unique_users_hidden.sort() 
unique_users_visible.sort() 

unique_users_hidden==unique_users_visible

False

#Songs per User
Filter Users available in Hidden Songs
Make Sure Users in Visible and Hidden dataset are same


In [0]:
df_valid_hidden=df_valid_hidden[df_valid_hidden['user_id'].isin(unique_users_visible)]

In [0]:
songs_per_user_visible=df_valid_visible.groupby(by=['user_id'])['song_id'].count()
df_no_songs_per_user_visible=pd.DataFrame(songs_per_user_visible)
df_no_songs_per_user_visible.describe()

,song_id
count,44365.000000
mean,1.799684
std,1.239261
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,16.000000


In [0]:
songs_per_user_hidden=df_valid_hidden.groupby(by=['user_id'])['song_id'].count()
df_no_songs_per_user_hidden=pd.DataFrame(songs_per_user_hidden)
df_no_songs_per_user_hidden.describe()

,song_id
count,28342.000000
mean,2.021205
std,1.355175
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,15.000000


#Filter Users in VISIBLE ( > n songs per user )

In [0]:
df_track_visible=df_no_songs_per_user_visible[df_no_songs_per_user_visible['song_id']>user_count]
df_filtered_visible=df_track_visible.reset_index()['user_id']
display(df_filtered_visible.head())
len(df_filtered_visible)

0    01047b99e00bb0743f66bb321bdd62e38fddb0a5
1    126ee2cc7dfb9bfb8e831633a5443a9b2b7490c0
2    1b8c08ef0705ff5b9329c175904e3c48d8f6b116
3    206bd0676fe110325996506f393538fbf422549e
4    291a37c97bedabb862b2f6bad7f439cf72123d84
Name: user_id, dtype: object

31

In [0]:
filtered_users_visible=df_valid_visible[df_valid_visible['user_id'].isin(df_filtered_visible)]
len(filtered_users_visible)
filtered_unique_songs_visible=filtered_users_visible['song_id'].unique()

In [0]:
filtered_users_visible.head()

,user_id,song_id,freq_song
5769,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SONNSYV12A8C146BEC,1
5773,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SODCADR12AF72A1A99,1
5774,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SODJWHY12A8C142CCE,3
5777,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SOJZFQA12AB017BFB0,1
5778,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SOPSYOY12A8C142E0B,1


In [0]:
filtered_users_visible.shape

(369, 3)

Filter hidden users based on filtered in visible users and their songs

In [0]:
filtered_users_hidden=df_valid_hidden[df_valid_hidden['user_id'].isin(df_filtered_visible)]
len(filtered_users_hidden)

277

In [0]:
#Check if both have same users but different songs
filtered_users_hidden['user_id'].unique()==filtered_users_visible['user_id'].unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [0]:
filtered_users_visible=filtered_users_visible[filtered_users_visible['user_id'].isin(filtered_users_hidden['user_id'])]

In [0]:
len(filtered_users_visible['user_id'].unique())

31

In [0]:
filtered_users_hidden['user_id'].unique()==filtered_users_visible['user_id'].unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [0]:
filtered_users_visible['user_id'].nunique()

31

In [0]:
filtered_users_hidden['user_id'].nunique()

31

In [0]:
# display(df_subset_full.columns)
filtered_users_visible.columns

Index(['user_id', 'song_id', 'freq_song'], dtype='object')

Merge Visible Songs with Metadata

In [0]:
lyrics_metadata.columns

df_visible=pd.merge(filtered_users_visible,lyrics_metadata,on=['song_id'],how='left')

display(df_visible.shape)

df_visible.to_csv("Datasets/filtered_users_visible_meta_data.csv")

(369, 16)

In [0]:
df_visible.head()

,user_id,song_id,freq_song,track_id,title,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SONNSYV12A8C146BEC,1,TRPFYYL128F92F7144,Float On,Float On,AR6AY7W1187B9AC569,a96ac800-bfcb-412a-8a63-0a98df600700,Modest Mouse,209.52771,0.808802,0.583858,2003,4906143,-1,0
1,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SODCADR12AF72A1A99,1,TRFGWOW12903CF14C3,Whataya Want From Me,Whataya Want From Me,ARIGTAO11FED0C4411,e942594a-91cc-427a-8beb-6815c28e9254,Adam Lambert,227.39546,0.756337,0.703292,2009,8622938,-1,0
2,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SODJWHY12A8C142CCE,3,TRVCUSW128F92F20C6,Hey_ Soul Sister,Save Me_ San Francisco,ARICCZI1187B9AEBB3,bfd5aec5-62a2-4560-a316-7c46c17babda,Train,216.76363,0.838886,0.783677,0,6786966,-1,0
3,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SOJZFQA12AB017BFB0,1,TREQNFM128F93165A8,Live Like We're Dying,Kris Allen,ARNPTKU11FED0C4418,9ce29a24-b118-41a3-9297-2618f26b124d,Kris Allen,212.50567,0.782033,0.531775,2009,6936569,-1,0
4,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SOPSYOY12A8C142E0B,1,TRTVTFT128F9327735,New Divide (Album Version),Transformers: Revenge Of The Fallen The Album,ARQUMH41187B9AF699,f59c5520-5f46-4d2c-b2c4-822eabf53419,Linkin Park,268.61669,0.887386,0.791143,2009,5566797,-1,0


In [0]:
lyrics_metadata.columns

df_hidden=pd.merge(filtered_users_hidden,lyrics_metadata,on='song_id',how='left')

display(df_hidden.shape)

df_hidden.to_csv("Datasets/filtered_users_hidden_meta_data.csv")

(277, 16)

In [0]:
df_visible.head()

,user_id,song_id,freq_song,track_id,title,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SONNSYV12A8C146BEC,1,TRPFYYL128F92F7144,Float On,Float On,AR6AY7W1187B9AC569,a96ac800-bfcb-412a-8a63-0a98df600700,Modest Mouse,209.52771,0.808802,0.583858,2003,4906143,-1,0
1,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SODCADR12AF72A1A99,1,TRFGWOW12903CF14C3,Whataya Want From Me,Whataya Want From Me,ARIGTAO11FED0C4411,e942594a-91cc-427a-8beb-6815c28e9254,Adam Lambert,227.39546,0.756337,0.703292,2009,8622938,-1,0
2,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SODJWHY12A8C142CCE,3,TRVCUSW128F92F20C6,Hey_ Soul Sister,Save Me_ San Francisco,ARICCZI1187B9AEBB3,bfd5aec5-62a2-4560-a316-7c46c17babda,Train,216.76363,0.838886,0.783677,0,6786966,-1,0
3,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SOJZFQA12AB017BFB0,1,TREQNFM128F93165A8,Live Like We're Dying,Kris Allen,ARNPTKU11FED0C4418,9ce29a24-b118-41a3-9297-2618f26b124d,Kris Allen,212.50567,0.782033,0.531775,2009,6936569,-1,0
4,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SOPSYOY12A8C142E0B,1,TRTVTFT128F9327735,New Divide (Album Version),Transformers: Revenge Of The Fallen The Album,ARQUMH41187B9AF699,f59c5520-5f46-4d2c-b2c4-822eabf53419,Linkin Park,268.61669,0.887386,0.791143,2009,5566797,-1,0


In [0]:
df_hidden.head()

,user_id,song_id,freq_song,track_id,title,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SOJJYDE12AF729FC16,1,TRAWVYP128F9305DF2,Two Is Better Than One,Love Drunk,ARG72Q21187FB36243,d6e65033-b56a-49df-8744-6fc3ffd75edd,Boys Like Girls featuring Taylor Swift,242.83383,0.852366,0.587033,2009,6265221,-1,0
1,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SOGSAYQ12AB018BA14,3,TRMEQQX12903CCD9D5,Eenie Meenie,Eenie Meenie,AR9LJ9D1187FB4303D,f7eb3be3-1065-4a53-94d4-e2d0c15c0b33,Sean Kingston and Justin Bieber,201.95220,0.925192,0.662876,0,8476464,-1,0
2,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SOJTLHS12A8C13F633,1,TRHEHYE128F42811F3,Ain't No Rest For The Wicked (Original Version),Cage The Elephant,AR12DMP11C8A41577B,b41b38d4-ef3e-4f37-8c75-cfe9af999696,Cage The Elephant,175.12444,0.754159,0.686729,2008,2946131,-1,0
3,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SOWKQYL12AB0183B15,1,TRLCLEM128F93402D3,Whatcha Say,Whatcha Say,ARICFET123E29C2452,6de0f914-3e60-4418-be3b-42e0feb6eb4d,Jason Derulo,222.61506,0.722760,0.702377,2009,5219250,-1,0
4,01047b99e00bb0743f66bb321bdd62e38fddb0a5,SOWGHHQ12A67021878,1,TRQVBSM128E078903A,Feel Good Inc (Album Version),Demon Days,ARAXU5Z1187B98FECD,e21857d5-3256-4547-afb3-4b6ded592596,Gorillaz,221.20444,0.775086,0.605191,2005,108811,-1,0


In [0]:
display(df_visible.shape)
display(df_hidden.shape)

(369, 16)

(277, 16)

#Add Topics to Filtered Songs

In [0]:
availableTracks_visible=set(df_visible['track_id'].unique())
availableTracks_hidden=set(df_hidden['track_id'].unique())
availableTracks_all=availableTracks_visible.union(availableTracks_hidden)
availableUsers=df_visible['user_id'].unique()

In [0]:
df_subLyrics = df_lyrics.loc[df_lyrics['track_id'].isin(availableTracks_all)]
df_subLyrics.shape

(17940, 5)

In [0]:
len(availableTracks_all)

173

In [0]:
stop_words = stopwords.words('english')
from string import punctuation
punc = list(set(punctuation))

stop_words = stop_words + punc

In [0]:
df_subLyrics['check'] = df_subLyrics['word'].apply(lambda x: x in stop_words)
df_subLyrics.head()

,track_id,mxm_tid,word,count,is_test,check
13142,TRAALAH128E078234A,727122,i,39,0,True
13143,TRAALAH128E078234A,727122,the,17,0,True
13144,TRAALAH128E078234A,727122,you,21,0,True
13145,TRAALAH128E078234A,727122,to,11,0,True
13146,TRAALAH128E078234A,727122,and,10,0,True


In [0]:
df_subLyrics = df_subLyrics[df_subLyrics['check'] == False]

del df_subLyrics['check']

In [0]:
#Creating tokenziner, lemmitizer, stemmer to process the review text
lemmatizer = WordNetLemmatizer()
porter_stemmer = PorterStemmer()

df_subLyrics['word'] = [lemmatizer.lemmatize(w) for w in df_subLyrics['word'].to_list()]
#df_subLyrics['word'] = [porter_stemmer.stem(w) for w in df_subLyrics['word'].to_list()]

df_subLyrics.shape

(11895, 5)

In [0]:
col = []

for i in range(len(df_subLyrics)):
    a = list((df_subLyrics.iloc[i,2],) * df_subLyrics.iloc[i,3])
    col.append(a)

    
df_subLyrics['bagWord'] = col
df_subLyrics.head()

,track_id,mxm_tid,word,count,is_test,bagWord
13162,TRAALAH128E078234A,727122,know,4,0,"[know, know, know, know]"
13169,TRAALAH128E078234A,727122,one,5,0,"[one, one, one, one, one]"
13171,TRAALAH128E078234A,727122,never,1,0,[never]
13173,TRAALAH128E078234A,727122,feel,1,0,[feel]
13174,TRAALAH128E078234A,727122,let,2,0,"[let, let]"


In [0]:
df_subLyrics_agg = df_subLyrics.groupby(by='track_id')['bagWord'].sum()
df_subLyrics_agg = df_subLyrics_agg.reset_index()
df_subLyrics_agg


,track_id,bagWord
0,TRAALAH128E078234A,"[know, know, know, know, one, one, one, one, o..."
1,TRABFDT12903CADD73,"[like, like, come, come, come, one, one, one, ..."
2,TRAEHHJ12903CF492F,"[love, love, know, oh, oh, oh, oh, oh, oh, oh,..."
3,TRAFQFV128F9326217,"[know, know, know, know, like, like, like, lik..."
4,TRAFUNV128F92CFEB2,"[like, like, like, like, like, go, one, one, o..."
...,...,...
168,TRZJNDO128EF343498,"[know, know, know, know, know, know, know, lik..."
169,TRZJNOD128F4264131,"[know, know, know, like, time, time, time, go,..."
170,TRZPSWK128F4243514,"[know, know, like, like, got, way, way, take, ..."
171,TRZXIRJ128E0781337,"[know, like, like, time, come, come, come, com..."


In [0]:
df_subLyrics_agg["doc"]=df_subLyrics_agg["bagWord"].apply(lambda x:" ".join(x))
df_subLyrics_agg

,track_id,bagWord,doc
0,TRAALAH128E078234A,"[know, know, know, know, one, one, one, one, o...",know know know know one one one one one never ...
1,TRABFDT12903CADD73,"[like, like, come, come, come, one, one, one, ...",like like come come come one one one one one o...
2,TRAEHHJ12903CF492F,"[love, love, know, oh, oh, oh, oh, oh, oh, oh,...",love love know oh oh oh oh oh oh oh oh oh one ...
3,TRAFQFV128F9326217,"[know, know, know, know, like, like, like, lik...",know know know know like like like like like l...
4,TRAFUNV128F92CFEB2,"[like, like, like, like, like, go, one, one, o...",like like like like like go one one one one se...
...,...,...,...
168,TRZJNDO128EF343498,"[know, know, know, know, know, know, know, lik...",know know know know know know know like like l...
169,TRZJNOD128F4264131,"[know, know, know, like, time, time, time, go,...",know know know like time time time go go oh oh...
170,TRZPSWK128F4243514,"[know, know, like, like, got, way, way, take, ...",know know like like got way way take take woul...
171,TRZXIRJ128E0781337,"[know, like, like, time, come, come, come, com...",know like like time come come come come come c...


In [0]:
doc_complete = df_subLyrics_agg.iloc[:,1]
doc_complete=doc_complete.tolist()

In [0]:
import gensim
from gensim import corpora
dictionary = corpora.Dictionary(doc_complete)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_complete]


In [0]:
Lda = gensim.models.ldamodel.LdaModel
# ldamodel = Lda(doc_term_matrix, id2word = dictionary, passes=50,per_word_topics=True)

lda_model = gensim.models.ldamodel.LdaModel(corpus=doc_term_matrix,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

text_data=df_subLyrics_agg.doc.values.tolist()
text_data_t=[[x] for x in text_data]

data_test = df_subLyrics_agg.doc.values.tolist()

doc_lda = lda_model[doc_term_matrix]

In [0]:
lda_model.print_topics()

[(0,
  '0.030*"like" + 0.023*"come" + 0.014*"got" + 0.013*"love" + 0.012*"get" + 0.010*"go" + 0.009*"think" + 0.009*"see" + 0.009*"almost" + 0.009*"let"'),
 (1,
  '0.026*"keep" + 0.019*"got" + 0.017*"man" + 0.015*"babi" + 0.013*"know" + 0.012*"time" + 0.012*"wait" + 0.012*"want" + 0.011*"well" + 0.011*"let"'),
 (2,
  '0.147*"da" + 0.036*"shot" + 0.021*"say" + 0.014*"whatcha" + 0.014*"chang" + 0.013*"master" + 0.010*"tie" + 0.010*"mm" + 0.010*"aint" + 0.010*"ca"'),
 (3,
  '0.099*"la" + 0.044*"que" + 0.024*"te" + 0.020*"gent" + 0.016*"de" + 0.016*"e" + 0.015*"tu" + 0.013*"en" + 0.012*"lieb" + 0.012*"ich"'),
 (4,
  '0.032*"oh" + 0.031*"girl" + 0.028*"take" + 0.024*"babi" + 0.023*"like" + 0.022*"yeah" + 0.021*"home" + 0.020*"na" + 0.020*"got" + 0.019*"want"'),
 (5,
  '0.031*"one" + 0.021*"know" + 0.017*"love" + 0.017*"oh" + 0.016*"time" + 0.013*"girl" + 0.013*"feel" + 0.012*"world" + 0.012*"want" + 0.011*"ca"'),
 (6,
  '0.052*"cri" + 0.041*"tonight" + 0.031*"one" + 0.026*"shake" + 0.025*"b

In [0]:
def format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=data_test):
    # Init output
    sent_topics_df = pd.DataFrame()
    data_rows=[]
    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = (row[0])
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        
        topic_contr={
            0:0,
            1:0,
            2:0,
            3:0,
            4:0,
            5:0,
            6:0,
            7:0,
            8:0,
            9:0
        }
        for j, (topic_num, prop_topic) in enumerate(row):
            # if j == 0:  # => dominant topic
              wp = ldamodel.show_topic(topic_num)
              topic_keywords = ", ".join([word for word, prop in wp])
              topic_contr[topic_num]=round(prop_topic,4)
        data_rows.append(topic_contr)
    contents = pd.Series(texts)
    sent_topics_df = pd.DataFrame(data_rows)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [0]:
df_topic_sents_keywords = format_topics_sentences()
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index(drop=True)
df_dominant_topic.columns = ['Topic_0','Topic_1','Topic_2','Topic_3','Topic_4','Topic_5','Topic_6','Topic_7','Topic_8','Topic_9','Text']
df_dominant_topic['track_id']=df_subLyrics_agg.iloc[:,0]

# # Show
# df_dominant_topic=df_dominant_topic.sort_values(by=['Dominant_Topic'])

display(df_dominant_topic)

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Text,track_id
0,0.0000,0.0000,0.7917,0.0,0.0000,0.1129,0.0241,0.0000,0.0000,0.0691,know know know know one one one one one never ...,TRAALAH128E078234A
1,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.9974,0.0000,like like come come come one one one one one o...,TRABFDT12903CADD73
2,0.0000,0.0000,0.0000,0.0,0.0000,0.9947,0.0000,0.0000,0.0000,0.0000,love love know oh oh oh oh oh oh oh oh oh one ...,TRAEHHJ12903CF492F
3,0.0000,0.0000,0.0000,0.0,0.7252,0.0000,0.0000,0.0000,0.0000,0.2723,know know know know like like like like like l...,TRAFQFV128F9326217
4,0.1984,0.0000,0.0000,0.0,0.0000,0.4131,0.2838,0.0000,0.0265,0.0760,like like like like like go one one one one se...,TRAFUNV128F92CFEB2
...,...,...,...,...,...,...,...,...,...,...,...,...
168,0.0000,0.0000,0.0000,0.0,0.0000,0.2791,0.0000,0.0391,0.5979,0.0820,know know know know know know know like like l...,TRZJNDO128EF343498
169,0.0221,0.0000,0.0103,0.0,0.8201,0.1302,0.0000,0.0000,0.0000,0.0163,know know know like time time time go go oh oh...,TRZJNOD128F4264131
170,0.0616,0.0000,0.0000,0.0,0.0000,0.9336,0.0000,0.0000,0.0000,0.0000,know know like like got way way take take woul...,TRZPSWK128F4243514
171,0.7489,0.0407,0.0000,0.0,0.0000,0.1054,0.0000,0.0780,0.0255,0.0000,know like like time come come come come come c...,TRZXIRJ128E0781337


In [0]:
df_train_features=pd.merge(df_dominant_topic,df_visible,on='track_id')
df_train_features.shape
df_train_features.to_csv('Datasets/final_train_songs_large.csv')

In [0]:
df_test_features=pd.merge(df_dominant_topic,df_hidden,on='track_id')
display(df_test_features.shape)
df_test_features.to_csv('Datasets/final_test_songs_large.csv')

(277, 27)

#Create Rating/Status for Test

Find Cross Multiplication between all user_id and track_id

In [0]:
lyrics_metadata.columns

Index(['track_id', 'title', 'song_id', 'release', 'artist_id', 'artist_mbid',
       'artist_name', 'duration', 'artist_familiarity', 'artist_hotttnesss',
       'year', 'track_7digitalid', 'shs_perf', 'shs_work'],
      dtype='object')

In [0]:
df_visible.count()

user_id               369
song_id               369
freq_song             369
track_id              369
title                 369
release               369
artist_id             369
artist_mbid           369
artist_name           369
duration              369
artist_familiarity    369
artist_hotttnesss     369
year                  369
track_7digitalid      369
shs_perf              369
shs_work              369
dtype: int64

In [0]:
df_visible['user_id'].nunique()

31

In [0]:
df_hidden['user_id'].nunique()

31

In [0]:
print(len(availableTracks_visible),len(availableTracks_hidden))

93 80


In [0]:
index_hidden = pd.MultiIndex.from_product([availableUsers,availableTracks_hidden], names = ["user_id","track_id"])
df_rating_hidden=pd.DataFrame(index = index_hidden).reset_index()

In [0]:
index_visible = pd.MultiIndex.from_product([availableUsers,availableTracks_visible], names = ["user_id","track_id"])
df_rating_visible=pd.DataFrame(index = index_visible).reset_index()

In [0]:
df_rating_hidden.shape

(2480, 2)

In [0]:
df_rating_visible.shape

(2883, 2)

Merge Cross Product dataframe to get song_metadata

In [0]:
mapping_songs_hidden=pd.merge(df_rating_hidden,lyrics_metadata,on=['track_id'],how='left')
display(mapping_songs_hidden.shape)

(2480, 15)

In [0]:
mapping_songs_visible=pd.merge(df_rating_visible,lyrics_metadata,on=['track_id'],how='left')
display(mapping_songs_visible.shape)

(2883, 15)

In [0]:
mapping_songs_hidden=pd.merge(df_dominant_topic,mapping_songs_hidden,on='track_id')
mapping_songs_visible=pd.merge(df_dominant_topic,mapping_songs_visible,on='track_id')

Merge Cross Product meta_data to for every user_id

In [0]:
df_final_hidden=pd.merge(mapping_songs_hidden,df_hidden.loc[:,['user_id','track_id','freq_song']],how='left',on=['user_id','track_id'])
df_final_hidden.shape

(2480, 27)

In [0]:
df_final_hidden.columns

Index(['Topic_0', 'Topic_1', 'Topic_2', 'Topic_3', 'Topic_4', 'Topic_5',
       'Topic_6', 'Topic_7', 'Topic_8', 'Topic_9', 'Text', 'track_id',
       'user_id', 'title', 'song_id', 'release', 'artist_id', 'artist_mbid',
       'artist_name', 'duration', 'artist_familiarity', 'artist_hotttnesss',
       'year', 'track_7digitalid', 'shs_perf', 'shs_work', 'freq_song'],
      dtype='object')

In [0]:
df_final_visible=pd.merge(mapping_songs_visible,df_visible.loc[:,['user_id','track_id','freq_song']],how='left',on=['user_id','track_id'])
df_final_visible.shape

(2883, 27)

#Add Unary Rating
Add Rating 0 if user hasn't listened to the song 
Add Rating 1 if user has listened to the song

In [0]:
display(df_final_hidden.head())
df_final_hidden['freq_song']=df_final_hidden['freq_song'].fillna(0)
df_final_hidden['status']=df_final_hidden.apply(lambda x: 1 if x['freq_song']>0 else 0,axis=1)


,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Text,track_id,user_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work,freq_song
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,01047b99e00bb0743f66bb321bdd62e38fddb0a5,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,126ee2cc7dfb9bfb8e831633a5443a9b2b7490c0,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,1b8c08ef0705ff5b9329c175904e3c48d8f6b116,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,206bd0676fe110325996506f393538fbf422549e,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,NaN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,291a37c97bedabb862b2f6bad7f439cf72123d84,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,NaN


In [0]:
display(df_final_hidden.head())
df_final_visible['freq_song']=df_final_visible['freq_song'].fillna(0)
df_final_visible['status']=df_final_visible.apply(lambda x: 1 if x['freq_song']>0 else 0,axis=1)

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Text,track_id,user_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work,freq_song,status
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,01047b99e00bb0743f66bb321bdd62e38fddb0a5,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,1.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,126ee2cc7dfb9bfb8e831633a5443a9b2b7490c0,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,1b8c08ef0705ff5b9329c175904e3c48d8f6b116,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,206bd0676fe110325996506f393538fbf422549e,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,291a37c97bedabb862b2f6bad7f439cf72123d84,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,0.0,0


In [0]:
display(df_final_hidden.head())
print(len(df_final_hidden))

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Text,track_id,user_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work,freq_song,status
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,01047b99e00bb0743f66bb321bdd62e38fddb0a5,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,1.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,126ee2cc7dfb9bfb8e831633a5443a9b2b7490c0,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,1b8c08ef0705ff5b9329c175904e3c48d8f6b116,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,206bd0676fe110325996506f393538fbf422549e,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9974,0.0,like like come come come one one one one one o...,TRABFDT12903CADD73,291a37c97bedabb862b2f6bad7f439cf72123d84,Up Up & Away,SOARUPP12AB01842E0,Man On The Moon: The End Of Day,ARD46C811C8A414F3F,e0e1db18-f7ba-4dee-95ff-7ae8cf545460,Kid Cudi,227.34322,0.808767,0.796314,2009,6394026,-1,0,0.0,0


2480


In [0]:
df_final_visible.head()
print(len(df_final_visible))

2883


In [0]:
df_final_hidden.to_csv('Datasets/final_test_topics_large.csv')

#Ratio


In [0]:
df_final_hidden['status'].value_counts()

0    2203
1     277
Name: status, dtype: int64

In [0]:
df_final_visible.to_csv('Datasets/final_train_topics_large.csv')

In [0]:
  df_final_visible['status'].value_counts()

0    2514
1     369
Name: status, dtype: int64

In [0]:
df_final_visible.head()

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Text,track_id,user_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work,freq_song,status
0,0.0,0.0,0.7917,0.0,0.0,0.1129,0.0241,0.0,0.0,0.0691,know know know know one one one one one never ...,TRAALAH128E078234A,01047b99e00bb0743f66bb321bdd62e38fddb0a5,Bitter Sweet Symphony,SOXQYSC12A6310E908,Bitter Sweet Symphony,AR0L04E1187B9AE90C,d4d17620-fd97-4574-92a8-a2cb7e72ce42,The Verve,360.25424,0.76773,0.58781,1997,9634,-1,0,0.0,0
1,0.0,0.0,0.7917,0.0,0.0,0.1129,0.0241,0.0,0.0,0.0691,know know know know one one one one one never ...,TRAALAH128E078234A,126ee2cc7dfb9bfb8e831633a5443a9b2b7490c0,Bitter Sweet Symphony,SOXQYSC12A6310E908,Bitter Sweet Symphony,AR0L04E1187B9AE90C,d4d17620-fd97-4574-92a8-a2cb7e72ce42,The Verve,360.25424,0.76773,0.58781,1997,9634,-1,0,0.0,0
2,0.0,0.0,0.7917,0.0,0.0,0.1129,0.0241,0.0,0.0,0.0691,know know know know one one one one one never ...,TRAALAH128E078234A,1b8c08ef0705ff5b9329c175904e3c48d8f6b116,Bitter Sweet Symphony,SOXQYSC12A6310E908,Bitter Sweet Symphony,AR0L04E1187B9AE90C,d4d17620-fd97-4574-92a8-a2cb7e72ce42,The Verve,360.25424,0.76773,0.58781,1997,9634,-1,0,0.0,0
3,0.0,0.0,0.7917,0.0,0.0,0.1129,0.0241,0.0,0.0,0.0691,know know know know one one one one one never ...,TRAALAH128E078234A,206bd0676fe110325996506f393538fbf422549e,Bitter Sweet Symphony,SOXQYSC12A6310E908,Bitter Sweet Symphony,AR0L04E1187B9AE90C,d4d17620-fd97-4574-92a8-a2cb7e72ce42,The Verve,360.25424,0.76773,0.58781,1997,9634,-1,0,0.0,0
4,0.0,0.0,0.7917,0.0,0.0,0.1129,0.0241,0.0,0.0,0.0691,know know know know one one one one one never ...,TRAALAH128E078234A,291a37c97bedabb862b2f6bad7f439cf72123d84,Bitter Sweet Symphony,SOXQYSC12A6310E908,Bitter Sweet Symphony,AR0L04E1187B9AE90C,d4d17620-fd97-4574-92a8-a2cb7e72ce42,The Verve,360.25424,0.76773,0.58781,1997,9634,-1,0,5.0,1


In [0]:
df_final_visible[df_final_visible.user_id=='3e154f655302a515efe03d9269f0843a012d60a2']


,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Text,track_id,user_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work,freq_song,status
6,0.0000,0.0000,0.7917,0.0,0.0000,0.1129,0.0241,0.0000,0.0000,0.0691,know know know know one one one one one never ...,TRAALAH128E078234A,3e154f655302a515efe03d9269f0843a012d60a2,Bitter Sweet Symphony,SOXQYSC12A6310E908,Bitter Sweet Symphony,AR0L04E1187B9AE90C,d4d17620-fd97-4574-92a8-a2cb7e72ce42,The Verve,360.25424,0.767730,0.587810,1997,9634,-1,0,0.0,0
37,0.0000,0.0000,0.0000,0.0,0.0000,0.9947,0.0000,0.0000,0.0000,0.0000,love love know oh oh oh oh oh oh oh oh oh one ...,TRAEHHJ12903CF492F,3e154f655302a515efe03d9269f0843a012d60a2,You're The One,SOBONKR12A58A7A7E0,If There Was A Way,AR5E44Z1187B9A1D74,0fb711af-c7ba-4bdc-b0b6-b8495fc0a590,Dwight Yoakam,239.30730,0.775370,0.514569,1990,6686507,-1,0,3.0,1
68,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.6081,0.0000,0.0000,0.3782,love love come go see let let let let let let ...,TRAOCBO12903D03670,3e154f655302a515efe03d9269f0843a012d60a2,Rianna,SONHWUN12AC468C014,The Update Collection Vol. 2,ARJIG5P1187B98D9DE,ea04c3a6-202e-4ca6-af56-53e09fd98fd2,Fisher,133.98159,0.657826,0.418252,0,6747439,-1,0,0.0,0
99,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.9867,0.0000,oh one one yeah yeah yeah yeah yeah yeah yeah ...,TRBGCAK128F425F3CC,3e154f655302a515efe03d9269f0843a012d60a2,Firestarter,SOHHJYE12A8AE47599,Put Your Hands Up! 4,AR4L4WQ1187FB51996,4a4ee089-93b1-4470-af9a-6ff575d32704,The Prodigy,225.09669,0.638614,0.601436,1996,3127217,6507,6507,0.0,0
130,0.1963,0.4673,0.0000,0.0,0.0000,0.3190,0.0000,0.0000,0.0000,0.0163,love love know know know like like like like l...,TRBKFKL128E078ED76,3e154f655302a515efe03d9269f0843a012d60a2,Hailie's Song,SOPCTBB12AF72A1B64,The Eminem Show,ARTH9041187FB43E1F,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,320.83546,0.871011,0.804830,2002,148227,-1,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,0.0247,0.0000,0.0000,0.0,0.1076,0.0000,0.0000,0.0000,0.0635,0.8016,love love love love know know go go go go go g...,TRZAVEC12903CD2DD2,3e154f655302a515efe03d9269f0843a012d60a2,Up,SOJUXGA12AC961885C,My Worlds,ARFCWSZ123526A0AFD,e0140a67-e4d1-4f13-8a01-364355bee46e,Justin Bieber,234.91873,0.839406,0.691249,2010,8497972,-1,0,0.0,0
2765,0.0000,0.0475,0.0000,0.0,0.0000,0.0000,0.0000,0.0730,0.4313,0.4457,love love love love love love love love love l...,TRZEDRT12903CD2DCC,3e154f655302a515efe03d9269f0843a012d60a2,Runaway Love,SODTHOY12AC9071AE7,My Worlds,ARFCWSZ123526A0AFD,e0140a67-e4d1-4f13-8a01-364355bee46e,Justin Bieber,212.53179,0.839406,0.691249,2010,8497966,-1,0,0.0,0
2796,0.5562,0.0974,0.0000,0.0,0.0000,0.0573,0.0375,0.0876,0.0000,0.1624,know know go one get get get get see let let l...,TRZGABY128F1462E00,3e154f655302a515efe03d9269f0843a012d60a2,Ocean Avenue,SOKQZYZ12A58A7767E,Ocean Avenue,AR048JZ1187B9AEB85,3630fff3-52fc-4e97-ab01-d68fd88e4135,Yellowcard,198.32118,0.872934,0.627467,2003,285605,-1,0,0.0,0
2827,0.0221,0.0000,0.0103,0.0,0.8201,0.1302,0.0000,0.0000,0.0000,0.0163,know know know like time time time go go oh oh...,TRZJNOD128F4264131,3e154f655302a515efe03d9269f0843a012d60a2,Paradise City,SOQGVCS12AF72A078D,Greatest Hits,ARM7EDF1187B9B3FA1,eeb1195b-f213-4ce1-b28c-8565211f8e43,Guns N' Roses,406.17751,0.844085,0.713670,1987,2234320,12880,12880,0.0,0


In [0]:
df_final_hidden[df_final_hidden.user_id=='206bd0676fe110325996506f393538fbf422549e']['status'].value_counts()

0    76
1     4
Name: status, dtype: int64